In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
import os
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from bc_video.utils import process_data, extract_from_raw_data, DIR_PATH
from lstm_utils import save_model
import random
import re


ValueError: libcublas.so.*[0-9] not found in the system path ['/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/bc_video', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/.venv/lib/python3.10/site-packages', '/media/chagan/2023-ELECTRA-field-MarcSchoolStudy']

In [5]:
train_test_split_ratio = 0.7
#features = ['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']
features = ['AU06_r','AU12_r']
extracted_data = extract_from_raw_data(features)

X_train, X_test, Y_train, Y_test, stats = process_data(extracted_data, features, train_test_split_ratio)

Data already exists in disk, reading..



In [6]:
for key in stats:
    print(key,':',stats[key])

Split ratio : 0.7022827812315975
IDs in Train : ['P318', 'P246', 'N332', 'N333', 'N253', 'P265', 'N251', 'P317', 'N254', 'P315', 'N249', 'N335', 'P240', 'P314', 'N334', 'P264', 'N255', 'N261', 'P316', 'P242', 'N262', 'P237', 'N260', 'P248']
IDs in Test : ['P243', 'P203', 'P263', 'N257', 'P244', 'N252', 'P239', 'N250', 'P245', 'N256', 'N258']
Number of Ns in train : 12
Number of Ps in train : 12
Number of Ns in test : 5
Number of Ps in test : 6


In [7]:
print('Train RB: ', extracted_data.loc[ extracted_data["pair_id"].isin([ d for d in stats['IDs in Train'] if d[0] == "P" ])].shape[0] / extracted_data.loc[ extracted_data['pair_id'].isin(stats['IDs in Train'])  ].shape[0] )
print('Test RB: ', extracted_data.loc[ extracted_data["pair_id"].isin([ d for d in stats['IDs in Test'] if d[0] == "P" ])].shape[0] / extracted_data.loc[ extracted_data['pair_id'].isin(stats['IDs in Test'])  ].shape[0] )

Train RB:  0.5500256780007718
Test RB:  0.5021422509376625


In [8]:
# class Classifier(nn.Module):
#     def __init__(self, num_of_features):
#         super().__init__()
#         self.input_block = nn.Sequential(nn.Linear(num_of_features, 5),
#                                     nn.BatchNorm1d(5),                  
#                                     nn.ReLU()                
#                                     ) 
#         self.hidden_block_1 = nn.Sequential(nn.Linear(5, 10),
#                                     nn.BatchNorm1d(10),                  
#                                     nn.ReLU()                
#                                     )
#         self.hidden_block_2 = nn.Sequential(nn.Linear(10, 10),
#                                     nn.BatchNorm1d(10),                  
#                                     nn.ReLU()                
#                                     )
#         self.output_block = nn.Sequential(nn.Linear(10, 1),
#                                     #nn.Sigmoid()
#         )

#     def forward(self, x):
#         x = self.input_block(x)
#         x = self.hidden_block_1(x)
#         x = self.hidden_block_2(x)
#         x = self.output_block(x)
#         return x

In [9]:
# Exp -3
# class Classifier(nn.Module):
#     def __init__(self, number_of_features):
#         super().__init__()
#         self.layer1 = nn.Linear(number_of_features, 10)                        
#         self.layer2 = nn.Linear(10, 10)
#         self.layer3 = nn.Linear(10, 1)

#         self.bc10 = nn.BatchNorm1d(10)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout1d(p=0.2)

#         nn.init.xavier_normal_(self.layer1.weight) 
#         nn.init.xavier_normal_(self.layer2.weight)
#         nn.init.xavier_normal_(self.layer3.weight)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.layer2(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.dropout(x)        
#         x = self.layer3(x)
        
#         return x

# Exp -4
# class Classifier(nn.Module):
#     def __init__(self, number_of_features):
#         super().__init__()
#         self.layer1 = nn.Linear(number_of_features, 100)                        
#         self.layer2 = nn.Linear(100, 50)
#         self.layer3 = nn.Linear(50, 10)
#         self.layer4 = nn.Linear(10, 1)

#         self.bc100 = nn.BatchNorm1d(100)
#         self.bc50 = nn.BatchNorm1d(50)
#         self.bc10 = nn.BatchNorm1d(10)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout1d(p=0.2)

#         nn.init.xavier_normal_(self.layer1.weight) 
#         nn.init.xavier_normal_(self.layer2.weight)
#         nn.init.xavier_normal_(self.layer3.weight)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.bc100(x)
#         x = self.relu(x)
#         x = self.layer2(x)
#         x = self.bc50(x)
#         x = self.relu(x)
#         x = self.dropout(x)        
#         x = self.layer3(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.layer4(x)
        
#         return x

In [10]:
# Exp -5
# class Classifier(nn.Module):
#     def __init__(self, number_of_features):
#         super().__init__()
#         self.layer1 = nn.Linear(number_of_features, 5)                        
#         self.layer2 = nn.Linear(5, 5)
#         self.layer3 = nn.Linear(5, 1)
#         #self.layer4 = nn.Linear(5, 1)

#         # self.bc100 = nn.BatchNorm1d(100)
#         # self.bc50 = nn.BatchNorm1d(50)
#         # self.bc10 = nn.BatchNorm1d(10)
#         self.bc5 = nn.BatchNorm1d(5)
#         self.relu = nn.ReLU()
#         #self.dropout = nn.Dropout1d(p=0.2)

#         nn.init.xavier_normal_(self.layer1.weight) 
#         nn.init.xavier_normal_(self.layer2.weight)
#         nn.init.xavier_normal_(self.layer3.weight)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.bc5(x)
#         x = self.relu(x)
#         x = self.layer2(x)
#         x = self.bc5(x)
#         x = self.relu(x)
#         x = self.layer3(x)
        
#         return x

# 2 AU approach Exp- 6
# class Classifier(nn.Module):
#     def __init__(self, number_of_features):
#         super().__init__()
#         self.layer1 = nn.Linear(number_of_features, 5)                        
#         self.layer2 = nn.Linear(5, 5)
#         self.layer3 = nn.Linear(5, 1)

#         self.bc10 = nn.BatchNorm1d(5)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout1d(p=0.2)

#         nn.init.xavier_normal_(self.layer1.weight) 
#         nn.init.xavier_normal_(self.layer2.weight)
#         nn.init.xavier_normal_(self.layer3.weight)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.layer2(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         #x = self.dropout(x)        
#         x = self.layer3(x)
        
#         return x

In [ ]:
#Exp - 7
class Classifier(nn.Module):
    def __init__(self, number_of_features):
        super().__init__()
        self.layer1 = nn.Linear(number_of_features, 5)                        
        self.layer2 = nn.Linear(5, 5)
        self.layer3 = nn.Linear(5, 1)
        #self.layer4 = nn.Linear(5, 1)

        # self.bc100 = nn.BatchNorm1d(100)
        # self.bc50 = nn.BatchNorm1d(50)
        # self.bc10 = nn.BatchNorm1d(10)
        self.bc5 = nn.BatchNorm1d(5)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout1d(p=0.2)

        nn.init.xavier_normal_(self.layer1.weight) 
        nn.init.xavier_normal_(self.layer2.weight)
        nn.init.xavier_normal_(self.layer3.weight)

    def forward(self, x):
        x = self.layer1(x)
        x = self.bc5(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.bc5(x)
        x = self.relu(x)
        x = self.layer3(x)
        
        return x

In [11]:
if torch.cuda.is_available():
    #print("RTX 4090 let's goo")
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

def predict(lr, ep, batch_len, wd, model, x_train, x_test, y_train, y_test, experiment_number, run):
    best_score = 0
    loss_function = nn.BCEWithLogitsLoss()
    learning_rate = lr  # 0.0005
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay= wd) #optim.SGD(model.parameters(), lr=learning_rate)
    df_train = {"Loss": [], "Accuracy": []}
    df_test = {"Loss": [], "Accuracy": []}
    model = model.to(device)
    epochs = ep  # 100
    batch_size = batch_len  # 256
    batch_start_train = torch.arange(0, len(x_train), batch_size)
    batch_start_test = torch.arange(0, len(x_test), batch_size)
    for epoch in range(epochs):
        print(f'\n Epoch: {epoch}')
        correct_train_preds = 0
        total_tr = 0
        train_loss_list = []
        model.train()
        with tqdm.tqdm(
            batch_start_train, 
            unit="batch", 
            mininterval=0, 
            disable=True, 
            desc="Train"
        ) as bar:
            bar.set_description(f"Train Epoch {epoch}")
            for start in bar:
                optimizer.zero_grad()

                x_batch_train = x_train[start : start + batch_size].to(device)
                y_batch_train = y_train[start : start + batch_size].to(device)
                #train_pred_raw = model(x_batch_train)  # forward pass .float()

                train_pred = model(x_batch_train)

                train_loss = loss_function(train_pred, y_batch_train)

                train_pred = train_pred.clamp(0,1)

                #_, train_pred = torch.max(train_pred.data, 1)
                
                train_loss_list.append(float(train_loss))  

                train_loss.backward()  # backward pass
                
                optimizer.step()  # update weights

                total_tr += y_batch_train.size(0)
                correct_train_preds += (train_pred.reshape(-1,1) == y_batch_train).sum().item() #correct_train_preds += (train_pred.round() == y_batch_train).sum().item()

        df_train["Loss"].append(np.mean(train_loss_list))
        df_train["Accuracy"].append(100 * correct_train_preds / total_tr)
        print(total_tr, correct_train_preds, correct_train_preds/total_tr, y_batch_train.shape)
        print(
            "Train Epoch [{}/{}], Accuracy: {:0.4f}; Loss: {:.4f}".format(
                epoch + 1,
                epochs,
                100 * correct_train_preds / total_tr,
                np.mean(train_loss_list),
            )
        )
        with torch.no_grad():
            model.eval()
            correct_test_preds = 0
            total_te = 0
            test_loss_list = []
            with tqdm.tqdm(
                batch_start_test,
                unit="batch",
                mininterval=0,
                disable=True,
                desc="Test"
            ) as bar1:
                bar1.set_description(f"Test Epoch {epoch}")
                for start in bar1:
                    x_batch_test = x_test[start : start + batch_size].to(device)
                    y_batch_test = y_test[start : start + batch_size].to(device)
                    #test_pred_raw = model(x_batch_test)
                    test_pred = model(x_batch_test)
                    total_te += y_batch_test.size(0)

                    test_loss = loss_function(test_pred, y_batch_test)
                    
                    test_pred = test_pred.clamp(0,1)
                    test_loss_list.append(float(test_loss))

                    correct_test_preds += (test_pred.reshape(-1,1) == y_batch_test).sum().item() #correct_test_preds += (test_pred.round() == y_batch_test).sum().item()
        
            df_test["Loss"].append(np.mean(test_loss_list))
            df_test["Accuracy"].append(100 * correct_test_preds / total_te)
            print(
                "Test Epoch [{}/{}], Accuracy: {:0.4f}; Loss: {:.4f}, CTP: {}".format(
                    epoch + 1,
                    epochs,
                    100 * correct_test_preds / total_te,
                    np.mean(test_loss_list),
                    correct_test_preds
                )
            )

            # print(ctp_df_merged.value_counts())
        if (100 * correct_test_preds / (len(x_test))) > best_score:
            best_score = 100 * correct_test_preds / (len(x_test))
            print('New best test metric observed! Saving Model..')

            if not os.path.exists(DIR_PATH / f'inference/mlp/experiment-{experiment_number}/models/run-{run}'):
                os.makedirs(DIR_PATH / f'inference/mlp/experiment-{experiment_number}/models/run-{run}')
                
            model_savepath = DIR_PATH / f'inference/mlp/experiment-{experiment_number}/models/run-{run}'
            save_model(model,epoch,optimizer,model_savepath )

    return df_train, df_test

In [12]:
num_of_features = X_train.shape[1]

Hyperparams = {
    "Learning rate": [0.1, 0.001, 0.0001], # , 0.05, 0.001, 0.005 , 0.001, 0.0001, 0.0005, 0.00001, 0.00005, 0.000001
    "Epochs": [500], #,, 100, 200 , 250, 500, 750
    "Batch size": [256], # , 64, 128 , 32, 64, 128
    "Weight decay": [1e-2, 1e-3, 1e-4]
}

Model = Classifier(num_of_features)

results = []
run = 0
experiment = 6

for lr in Hyperparams["Learning rate"]:
    for ep in Hyperparams["Epochs"]:
        for bs in Hyperparams["Batch size"]:
            for wd in Hyperparams["Weight decay"]:
                dataframe_train, dataframe_test = predict(
                    lr, ep, bs, wd, Model, X_train, X_test, Y_train, Y_test, experiment, run
                )
                results.append(
                    {
                        "Iteration": run,
                        "Learning rate": lr,
                        "Epoch": ep,
                        "Batch size": bs,
                        "Train acc": dataframe_train["Accuracy"],
                        "Train loss": dataframe_train["Loss"],
                        "Test acc": dataframe_test["Accuracy"],
                        "Test loss": dataframe_test["Loss"],
                        "Final Train acc": dataframe_train["Accuracy"][-1],
                        "Final Test acc": dataframe_test["Accuracy"][-1],
                    }
                )
                if not os.path.exists(DIR_PATH / f'inference/mlp/experiment-{experiment}/data'):
                    os.makedirs(DIR_PATH / f'inference/mlp/experiment-{experiment}/data')
                
                save_path = DIR_PATH / f'inference/mlp/experiment-{experiment}/data/Run - {run}_{lr}_{ep}_{bs}.csv'
                
                results_df=pd.DataFrame()
                results_df['Train acc'] = dataframe_train["Accuracy"]
                results_df['Test acc'] = dataframe_test["Accuracy"]
                results_df['Train loss'] = dataframe_train["Loss"]
                results_df['Test loss'] = dataframe_test["Loss"]
                results_df.to_csv(save_path, index=False)
                run += 1
                print("Run:{0} ; Train acc: {1} ; Test acc: {2} for ( {3},{4},{5} )".format(
                    run,
                    dataframe_train["Accuracy"][-1],
                    dataframe_test["Accuracy"][-1],
                    lr,
                    ep,
                    bs
                ))


 Epoch: 0
344653 342190 0.992853681819105 torch.Size([77, 1])
Train Epoch [1/500], Accuracy: 99.2854; Loss: 0.0217
Test Epoch [1/500], Accuracy: 50.2142; Loss: 2.2967, CTP: 73367
New best test metric observed! Saving Model..

 Epoch: 1
344653 341786 0.9916814883375453 torch.Size([77, 1])
Train Epoch [2/500], Accuracy: 99.1681; Loss: 0.0414
Test Epoch [2/500], Accuracy: 50.2142; Loss: 2.2965, CTP: 73367

 Epoch: 2
344653 341530 0.9909387122700223 torch.Size([77, 1])
Train Epoch [3/500], Accuracy: 99.0939; Loss: 0.0375
Test Epoch [3/500], Accuracy: 50.2142; Loss: 2.2967, CTP: 73367

 Epoch: 3
344653 340762 0.9887103840674534 torch.Size([77, 1])
Train Epoch [4/500], Accuracy: 98.8710; Loss: 0.0566
Test Epoch [4/500], Accuracy: 50.2142; Loss: 2.2971, CTP: 73367

 Epoch: 4
344653 340762 0.9887103840674534 torch.Size([77, 1])
Train Epoch [5/500], Accuracy: 98.8710; Loss: 0.0480
Test Epoch [5/500], Accuracy: 50.2142; Loss: 1.7060, CTP: 73367

 Epoch: 5
344653 339482 0.9849965037298384 torch.

In [13]:
# Grid search results
max_test_accuracy = max(results, key=lambda x: x["Final Test acc"])
max_test_accuracy

{'Iteration': 0,
 'Learning rate': 0.1,
 'Epoch': 500,
 'Batch size': 256,
 'Train acc': [99.2853681819105,
  99.16814883375453,
  99.09387122700223,
  98.87103840674534,
  98.87103840674534,
  98.49965037298384,
  98.94531601349763,
  98.49965037298384,
  99.09387122700223,
  99.01959362024994,
  99.09387122700223,
  99.09387122700223,
  99.17540250628893,
  99.16814883375453,
  99.16814883375453,
  98.42537276623155,
  99.01959362024994,
  99.01959362024994,
  99.01959362024994,
  99.01959362024994,
  99.09387122700223,
  99.09387122700223,
  99.16814883375453,
  99.16814883375453,
  99.16814883375453,
  98.87103840674534,
  98.87103840674534,
  98.87103840674534,
  99.09387122700223,
  99.09387122700223,
  99.16814883375453,
  99.09387122700223,
  99.09387122700223,
  99.16814883375453,
  99.09387122700223,
  99.16814883375453,
  99.16814883375453,
  99.09387122700223,
  98.87103840674534,
  99.01959362024994,
  99.01959362024994,
  98.87103840674534,
  99.01959362024994,
  99.09387

In [30]:
max_train_accuracy = max(results, key=lambda x: x["Train acc"])
max_train_accuracy

{'Iteration': 23,
 'Learning rate': 0.005,
 'Epoch': 1500,
 'Batch size': 256,
 'Train acc': [81.07706413445928,
  82.67604564514056,
  82.10256140719717,
  80.57687141572352,
  81.23620309050773,
  80.18705397292594,
  75.70429937979607,
  57.77999696321992,
  49.894588691498186,
  48.84806755667683,
  57.209140708036955,
  72.39508508824183,
  77.05041054930679,
  81.17108751766588,
  81.40176600441501,
  80.17741803613768,
  81.84735508135067,
  81.68996811380917,
  80.2156697852062,
  81.3585502879101,
  80.92931310370604,
  79.62028569092587,
  79.71868904539986,
  82.11453332866137,
  82.51223471974023,
  81.76559561769275,
  81.1503556536669,
  81.24437903687353,
  81.42512585117441,
  81.6575563264305,
  82.72889729843372,
  80.83470572433045,
  82.20709672144551,
  81.39534204655618,
  82.87489634068001,
  82.78291694406485,
  82.53793055117558,
  81.73551981499001,
  76.70234883259165,
  79.39953513904949,
  81.00990457502598,
  79.05556139551724,
  81.0128245558709,
  81.475

In [2]:
import re 

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [14]:
experiment_number = 6
data_path = DIR_PATH / f'inference/mlp/experiment-{experiment_number}/data'
plot_path = DIR_PATH / f'inference/mlp/experiment-{experiment_number}/plots'
if not os.path.exists(plot_path):
    os.makedirs(plot_path)
test_acc_df = pd.DataFrame(columns=['Run', 'Final test acc'])
csvs = [i.name for i in Path(data_path).glob('*.csv')]
csvs = sorted_nicely(csvs)
max_acc = 0.0
max_acc_run = str()
for k in csvs:
    
    #if results.index(k) == 27:
    fig,axs=plt.subplots(2,2)
    fig.suptitle(k[:-4])
    fig.tight_layout(pad=2.0)
    dk = pd.read_csv(data_path / k)
    axs[0, 0].plot(dk["Train loss"].astype(float).values)
    axs[0, 0].set_title('Train loss')
    axs[0, 0].set(ylabel='Loss')
    axs[0, 1].plot(dk["Train acc"], 'tab:red') #.apply( lambda x: float(x[7:-1])).values, 'tab:red')
    axs[0, 1].set_title('Train acc')
    axs[1, 0].plot(dk["Test loss"].astype(float).values, 'tab:green')
    test_acc = dk["Test acc"].values[-1] #.apply( lambda x: float(x[7:-1])).values[-1]
    #print(test_acc)
    run = k[:-4]
    test_acc_df.loc[len(test_acc_df)] = [run, test_acc]

    if test_acc > max_acc:
        max_acc = test_acc
        max_acc_run = run
    axs[1, 0].set_title('Test loss')
    axs[1, 0].set(xlabel='Epochs', ylabel='Loss')
    axs[1, 1].plot(dk["Test acc"], 'tab:orange') #.apply( lambda x: float(x[7:-1])).values, 'tab:orange')
    axs[1, 1].set_title('Test acc')
    axs[1, 1].set(xlabel='Epochs')
    #plt.close()
    if os.path.exists(plot_path /  f'{k[:-4]}.png'):
        plt.close()
        continue
    else:
        #continue
        plt.savefig(plot_path / f'{k[:-4]}.png')
        plt.close()

In [29]:
test_acc_df.sort_values(by='Final test acc', ascending=False)
#2

,Run,Final test acc
2,Run - 2_0.001_250_256,60.912988
15,Run - 15_0.005_500_512,60.912988
12,Run - 12_0.005_250_512,60.912988
8,Run - 8_0.001_1000_256,60.912988
20,Run - 20_0.005_1000_256,60.912988
17,Run - 17_0.005_500_256,60.912988
21,Run - 21_0.005_1500_512,60.912306
5,Run - 5_0.001_500_256,60.887047
23,Run - 23_0.005_1500_256,60.874077
47,Run - 47_0.0005_1500_256,60.237838


In [27]:
test_acc_df.sort_values(by='Final test acc', ascending=False)
#3

,Run,Final test acc
12,Run - 12_0.005_250_512,47.824240
14,Run - 14_0.005_250_256,47.824240
36,Run - 36_0.0005_250_512,47.824240
41,Run - 41_0.0005_500_256,47.824240
44,Run - 44_0.0005_1000_256,47.824240
23,Run - 23_0.005_1500_256,47.824240
42,Run - 42_0.0005_1000_512,47.824240
47,Run - 47_0.0005_1500_256,47.824240
38,Run - 38_0.0005_250_256,47.824240
39,Run - 39_0.0005_500_512,47.824240


In [23]:
test_acc_df.sort_values(by='Final test acc', ascending=False)
#4

,Run,Final test acc
0,Run - 0_0.001_1000_512,48.027532
1,Run - 1_0.001_1000_1024,48.027532
2,Run - 2_0.001_1000_256,48.027532
3,Run - 3_0.001_1500_512,48.027532
4,Run - 4_0.001_1500_1024,48.027532
5,Run - 5_0.001_1500_256,48.027532
6,Run - 6_0.0001_1000_512,48.027532
7,Run - 7_0.0001_1000_1024,48.027532
8,Run - 8_0.0001_1000_256,48.027532
9,Run - 9_0.0001_1500_512,48.027532
